In [54]:
import pandas as pd
import numpy as np
import math
from sklearn import metrics

In [51]:
def log_meas(current):
    new_meas = 1 - math.log(current, 50000.0)
    if new_meas > 1: new_meas = 1
    if new_meas < 0: new_meas = 0
    return new_meas

def log_df(data):
    new_data = data.copy(deep=True)
    for i in new_data.columns:
        new_data[i] = [log_meas(j) for j in new_data[i]]
    return new_data

In [52]:
performance_data = pd.read_csv("models` performance.csv")
performance_data = performance_data[performance_data.length == 9].drop(["Unnamed: 0", "sequence", "length"], axis = 1)

In [53]:
logged_perf = log_df(performance_data)
logged_perf.tail()

,meas,netmhc,netmhcpan,smmpmbec_cpp
27675,0.747391,0.434557,0.605658,0.567139
27676,0.022339,0.293037,0.202166,0.371139
27677,0.028511,0.283247,0.148324,0.273245
27678,0.141940,0.079586,0.195356,0.143642
27679,0.535430,0.496485,0.389441,0.569267


In [57]:
print(metrics.mean_squared_error(logged_perf.netmhc, logged_perf.meas))

0.0261240093427


In [59]:
print(metrics.mean_squared_error(logged_perf.netmhcpan, logged_perf.meas))

0.0251776608622


In [58]:
print(metrics.mean_squared_error(logged_perf.smmpmbec_cpp, logged_perf.meas))

0.0300394502946
